<a href="https://colab.research.google.com/github/NajimovOtabek/Assignment_SW/blob/main/Binary_UNSW_NB15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown --id 1AaFPfmUsKhplxHukzQ0k7DKZgEUiDV4A
!gdown --id 1IlPVdYv0gGZFJxNclcbUf46vHrWhLWeS

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1AaFPfmUsKhplxHukzQ0k7DKZgEUiDV4A
To: /content/UNSW_NB15_training-set.csv
100% 15.4M/15.4M [00:00<00:00, 82.5MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1IlPVdYv0gGZFJxNclcbUf46vHrWhLWeS
To: /content/UNSW_NB15_testing-set.csv
100% 32.3M/32.3M [00:00<00:00, 38.5MB/s]


In [2]:
!wget https://raw.githubusercontent.com/Jumabek/net_intrusion_detection/develop/preprocessing.py

--2023-10-06 10:42:24--  https://raw.githubusercontent.com/Jumabek/net_intrusion_detection/develop/preprocessing.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3790 (3.7K) [text/plain]
Saving to: ‘preprocessing.py’

preprocessing.py    100%[===================>]   3.70K  --.-KB/s    in 0s      

2023-10-06 10:42:24 (62.7 MB/s) - ‘preprocessing.py’ saved [3790/3790]



In [3]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, balanced_accuracy_score, classification_report
from preprocessing import read_data, load_data, normalize
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
from sklearn.preprocessing import LabelEncoder, Normalizer
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE

In [4]:
data_train = pd.read_csv("UNSW_NB15_training-set.csv")
data_test = pd.read_csv("UNSW_NB15_testing-set.csv")

### Dropping the 'service' column due to a high number of missing (None) values, making it less useful for analysis.


In [5]:
data_train = data_train.drop(columns="service")
data_test = data_test.drop(columns="service")

In [6]:
data = pd.concat([data_train, data_test], ignore_index=True)

In [7]:
data_train = data_train.drop(columns="attack_cat")
data_test = data_test.drop(columns="attack_cat")

In [8]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

def make_value2index(attacks):
    attacks = sorted(attacks)
    return {attack: i for i, attack in enumerate(attacks)}

# changes label from string to integer/index
def encode_label(Y_str):
    labels_d = make_value2index(np.unique(Y_str))
    Y = np.array([labels_d[attack] for attack in Y_str])
    return Y

def normalize(data):
    data = data.astype(np.float32)
    eps = 1e-15
    mask = data == -1
    data[mask] = 0
    mean_i = np.mean(data, axis=0)
    min_i = np.min(data, axis=0)
    max_i = np.max(data, axis=0)
    r = max_i - min_i + eps
    data = (data - mean_i) / r
    data[mask] = 0
    return data

def load_data(data):
    num_records, num_features = data.shape  # Use shape instead of size()
    print(f"There are {num_records} flow records with {num_features} feature dimensions.")


    # Strip the whitespace from column names
    data = data.rename(columns=lambda x: x.strip())
    print("Stripped column names.")

    # Handle missing values (NaN)
    nan_count = data.isnull().sum().sum()
    print(f"There are {nan_count} NaN entries.")
    if nan_count > 0:
        data.fillna(data.mean(), inplace=True)
        print("Filled NaN entries with column mean.")

    # Convert 'proto' and 'state' columns to numeric using Label Encoding, if they exist
    if 'proto' in data.columns and 'state' in data.columns:
        label_encoder = LabelEncoder()
        data['proto'] = label_encoder.fit_transform(data['proto'])
        data['state'] = label_encoder.fit_transform(data['state'])
        #data['attack_cat'] = label_encoder.fit_transform(data['attack_cat'])
        print("Converted 'proto' and 'state' columns to numeric using Label Encoding.")

    # Convert all columns to float type
    data = data.astype(float)
    print("Converted all columns to numeric types.")

    # Ensure there are no NaN values
    assert data.isnull().sum().sum() == 0, "There should not be any NaN values."
    return (data)


In [9]:
data_train = load_data(data_train)
data_test = load_data(data_test)

There are 82332 flow records with 43 feature dimensions.
Stripped column names.
There are 0 NaN entries.
Converted 'proto' and 'state' columns to numeric using Label Encoding.
Converted all columns to numeric types.
There are 175341 flow records with 43 feature dimensions.
Stripped column names.
There are 0 NaN entries.
Converted 'proto' and 'state' columns to numeric using Label Encoding.
Converted all columns to numeric types.


In [10]:
X_train = data_train.iloc[:, :-1].values
X_train = normalize(X_train)
y_train = data_train.iloc[:, -1].values
X_test = data_test.iloc[:, :-1].values
X_test = normalize(X_test)
y_test = data_test.iloc[:, -1].values

In [11]:
unique_before, counts_before = np.unique(y_train, return_counts=True)
print("Class distribution before balancing:", dict(zip(unique_before, counts_before)))
unique_before, counts_before = np.unique(y_test, return_counts=True)
print("Class distribution before balancing:", dict(zip(unique_before, counts_before)))

Class distribution before balancing: {0.0: 37000, 1.0: 45332}
Class distribution before balancing: {0.0: 56000, 1.0: 119341}


In [12]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [13]:
y_pred = clf.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print('Classification Report:')
print(classification_report(y_test, y_pred))


Accuracy: 0.6815405410029599
Classification Report:
              precision    recall  f1-score   support

         0.0       0.50      0.84      0.63     56000
         1.0       0.89      0.61      0.72    119341

    accuracy                           0.68    175341
   macro avg       0.70      0.72      0.67    175341
weighted avg       0.77      0.68      0.69    175341



In [14]:
from sklearn.metrics import brier_score_loss

# Get predicted probabilities
y_prob = clf.predict_proba(X_test)[:, 1]  # Taking the probabilities for the positive class (usually index 1)

# Calculate Brier Score
brier_score = brier_score_loss(y_test, y_prob)
print(f"Brier Score: {brier_score}")


Brier Score: 0.23450784578755282


In [15]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc

# ROC AUC
y_pred_proba = clf.predict_proba(X_test)[:, 1]  # Probability estimates of the positive class
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f'ROC AUC: {roc_auc}')

# PR AUC
precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
pr_auc = auc(recall, precision)
print(f'PR AUC: {pr_auc}')

ROC AUC: 0.6673924480510232
PR AUC: 0.845144408886913
